<a href="https://colab.research.google.com/github/nberonio/Shawnee-Prescribed-Fire/blob/main/Shawnee2_PlanetHistogramMatch_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.1 MB/s eta 0:00:00


In [ ]:
import ee
import geemap

In [ ]:
ee.Authenticate()
ee.Initialize(project='sixth-firmament-425715-j4')

In [ ]:
def lookup(source_hist, target_hist):
    """Creates a lookup table to make a source histogram match a target histogram.

    Args:
        source_hist: The histogram to modify. Expects the Nx2 array format produced by ee.Reducer.autoHistogram.
        target_hist: The histogram to match to. Expects the Nx2 array format produced by ee.Reducer.autoHistogram.

    Returns:
        A dictionary with 'x' and 'y' properties that respectively represent the x and y
        array inputs to the ee.Image.interpolate function.
    """

    # Split the histograms by column and normalize the counts.
    source_values = source_hist.slice(1, 0, 1).project([0])
    source_counts = source_hist.slice(1, 1, 2).project([0])
    source_counts = source_counts.divide(source_counts.get([-1]))

    target_values = target_hist.slice(1, 0, 1).project([0])
    target_counts = target_hist.slice(1, 1, 2).project([0])
    target_counts = target_counts.divide(target_counts.get([-1]))

    # Find first position in target where targetCount >= srcCount[i], for each i.
    def make_lookup(n):
        return target_values.get(target_counts.gte(n).argmax())

    lookup = source_counts.toList().map(make_lookup)

    return {'x': source_values.toList(), 'y': lookup}

In [ ]:
def histogram_match(source_img, target_img, geometry):
    """Performs histogram matching for 3-band RGB images by forcing the histogram CDF of source_img to match target_img.

    Args:
        source_img: A 3-band ee.Image to be color matched. Must have bands named 'R', 'G', and 'B'.
        target_img: A 3-band ee.Image for color reference. Must have bands named 'R', 'G', and 'B'.
        geometry: An ee.Geometry that defines the region to generate RGB histograms for.
          It should intersect both source_img and target_img inputs.

    Returns:
        A copy of src_img color-matched to target_img.
    """

    args = {
        'reducer': ee.Reducer.autoHistogram(maxBuckets=256, cumulative=True),
        'geometry': geometry,
        'scale': 1, # Need to specify a scale, but it doesn't matter what it is because bestEffort is true.
        'maxPixels': 65536 * 4 - 1,
        'bestEffort': True
    }

    # Only use pixels in target that have a value in source (inside the footprint and unmasked).
    source = source_img.reduceRegion(**args)
    target = target_img.updateMask(source_img.mask()).reduceRegion(**args)

    return ee.Image(ee.Image.cat(
        source_img.select(['R']).interpolate(**lookup(source.getArray('R'), target.getArray('R'))),
        source_img.select(['G']).interpolate(**lookup(source.getArray('G'), target.getArray('G'))),
        source_img.select(['B']).interpolate(**lookup(source.getArray('B'), target.getArray('B'))),
        source_img.select(['nir']).interpolate(**lookup(source.getArray('nir'), target.getArray('nir')))
    ).copyProperties(source_img, ['system:time_start']))

In [ ]:
def find_closest(target_image, image_col, days):
    """Filter images in a collection by date proximity and spatial intersection to a target image.

    Args:
        target_image: An ee.Image whose observation date is used to find near-date images in
          the provided image_col image collection. It must have a 'system:time_start' property.
        image_col: An ee.ImageCollection to filter by date proximity and spatial intersection
          to the target_image. Each image in the collection must have a 'system:time_start'
          property.
        days: A number that defines the maximum number of days difference allowed between
          the target_image and images in the image_col.

    Returns:
        An ee.ImageCollection that has been filtered to include those images that are within the
          given date proximity to target_image and intersect it spatially.
    """

    # Compute the timespan for N days (in milliseconds).
    range = ee.Number(days).multiply(1000 * 60 * 60 * 24)

    filter = ee.Filter.And(
        ee.Filter.maxDifference(range, 'system:time_start', None, 'system:time_start'),
        ee.Filter.intersects('.geo', None, '.geo'))

    closest = (ee.Join.saveAll('matches', 'measure')
        .apply(ee.ImageCollection([target_image]), image_col, filter))

    return ee.ImageCollection(ee.List(closest.first().get('matches')))

In [ ]:
feature_collection_Ashby1 = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/Ashby1_geometry')
geometry_Ashby1 = feature_collection_Ashby1.geometry()
feature_collection_Blowdown = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/Blowdown_geometry_2')
geometry_Blowdown = feature_collection_Blowdown.geometry()
feature_collection_JohnsonCo = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/JohnsonCo_geometry')
geometry_JohnsonCo = feature_collection_JohnsonCo.geometry()
feature_collection_LaRuePineHills = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/LaRuePineHills_geometry')
geometry_LaRuePineHills = feature_collection_LaRuePineHills.geometry()
feature_collection_LeeMine = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/LeeMine_geometry')
geometry_LeeMine = feature_collection_LeeMine.geometry()
feature_collection_McConnel1 = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/McConnel1_geometry')
geometry_McConnel1 = feature_collection_McConnel1.geometry()
feature_collection_McConnel2 = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/McConnel2_geometry')
geometry_McConnel2 = feature_collection_McConnel2.geometry()
feature_collection_OzarkHill = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/OzarkHill_geometry')
geometry_OzarkHill = feature_collection_OzarkHill.geometry()
feature_collection_PennantBar1 = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/PennantBar1_geometry')
geometry_PennantBar1 = feature_collection_PennantBar1.geometry()
feature_collection_SimpsonTownshipBarrens_PVT = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/SimpsonTownshipBarrens_PVT_geometry')
geometry_SimpsonTownshipBarrens_PVT = feature_collection_SimpsonTownshipBarrens_PVT.geometry()
feature_collection_TPL = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/TPL_geometry')
geometry_TPL = feature_collection_TPL.geometry()
feature_collection_White = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/White_geometry_2')
geometry_White = feature_collection_White.geometry()
feature_collection_WhoopieCat = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/WhoopieCat_geometry')
geometry_WhoopieCat = feature_collection_WhoopieCat.geometry()

Ashby1_20190209_Date=ee.Date.fromYMD(2019, 2, 9).millis()
Blowdown_20210122_2_Date=ee.Date.fromYMD(2021, 1, 22).millis()
Blowdown_20210205_2_Date=ee.Date.fromYMD(2021, 2, 5).millis()
Blowdown_20220219_2_Date=ee.Date.fromYMD(2022, 2, 19 ).millis()
JohnsonCo_20170216_2_Date=ee.Date.fromYMD(2017, 2, 16).millis()
JohnsonCo_20170216_3_Date=ee.Date.fromYMD(2017, 2, 16).millis()
JohnsonCo_20170308_Date=ee.Date.fromYMD(2017, 3, 8).millis()
LaRuePineHills_20170217_Date=ee.Date.fromYMD(2017, 2, 17).millis()
LaRue_20170315_2_Date=ee.Date.fromYMD(2017, 3, 15).millis()
LaRue_20180306_2_Date=ee.Date.fromYMD(2018, 3, 6).millis()
LeeMine_20200221_Date=ee.Date.fromYMD(2020, 2, 21).millis()
LeeMine_20200307_2_Date=ee.Date.fromYMD(2020, 3, 7).millis()
LeeMine_20210304_Date=ee.Date.fromYMD(2021, 3, 4).millis()
McConnel1_20190316_Date=ee.Date.fromYMD(2019, 3, 16).millis()
McConnel2_20200307_Date=ee.Date.fromYMD(2020, 3, 7).millis()
McConnel2_20210303_Date=ee.Date.fromYMD(2021, 3, 3).millis()
OzarkHill_20170217_2_Date=ee.Date.fromYMD(2017, 2, 17).millis()
OzarkHill_20170407_Date=ee.Date.fromYMD(2017, 4, 7).millis()
OzarkHill_20180227_Date=ee.Date.fromYMD(2018, 2, 27).millis()
PennantBar1_20170208_2_Date=ee.Date.fromYMD(2017, 2, 8).millis()
PennantBar1_20170216_Date=ee.Date.fromYMD(2017, 2, 16).millis()
PennantBar1_20180212_Date=ee.Date.fromYMD(2018, 2, 12).millis()
SimpsonTownshipBarrens_PVT_20200229_Date=ee.Date.fromYMD(2020, 2, 29).millis()
TPL_20200119_Date=ee.Date.fromYMD(2020, 1, 19).millis()
TPL_20200227_Date=ee.Date.fromYMD(2020, 2, 27).millis()
TPL_20210205_Date=ee.Date.fromYMD(2021, 2, 5).millis()
White_20180227_Date=ee.Date.fromYMD(2018, 2, 27).millis()
White_20180306_Date=ee.Date.fromYMD(2018, 3, 6).millis()
White_20190322_2_Date=ee.Date.fromYMD(2019, 3, 22).millis()
WhoopieCat_20200307_2_Date=ee.Date.fromYMD(2020, 3, 7).millis()


Ashby1_20190209 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Ashby_20190209')
    .clip(geometry_Ashby1).set('system:time_start',Ashby1_20190209_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Blowdown_20210122_2 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Blowdown_20210122_2')
    .clip(geometry_Blowdown).set('system:time_start',Blowdown_20210122_2_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Blowdown_20210205_2 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Blowdown_20210205_2')
    .clip(geometry_Blowdown).set('system:time_start',Blowdown_20210205_2_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Blowdown_20220219_2 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Blowdown_20220219_2')
    .clip(geometry_Blowdown).set('system:time_start',Blowdown_20220219_2_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
JohnsonCo_20170216_2 = (ee.Image('projects/sixth-firmament-425715-j4/assets/JohnsonCo_20170216_2')
    .clip(geometry_JohnsonCo).set('system:time_start',JohnsonCo_20170216_2_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
JohnsonCo_20170216_3 = (ee.Image('projects/sixth-firmament-425715-j4/assets/JohnsonCo_20170216_3')
    .clip(geometry_JohnsonCo).set('system:time_start',JohnsonCo_20170216_3_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
JohnsonCo_20170308 = (ee.Image('projects/sixth-firmament-425715-j4/assets/JohnsonCo_20170308')
    .clip(geometry_JohnsonCo).set('system:time_start',JohnsonCo_20170308_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
LaRuePineHills_20170217 = (ee.Image('projects/sixth-firmament-425715-j4/assets/LaRuePineHills_20170217')
    .clip(geometry_LaRuePineHills).set('system:time_start',LaRuePineHills_20170217_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
LaRue_20170315_2 = (ee.Image('projects/sixth-firmament-425715-j4/assets/LaRue_20170315_2')
    .clip(geometry_LaRuePineHills).set('system:time_start',LaRue_20170315_2_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
LaRue_20180306_2 = (ee.Image('projects/sixth-firmament-425715-j4/assets/LaRue_20180306_2')
    .clip(geometry_LaRuePineHills).set('system:time_start',LaRue_20180306_2_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
LeeMine_20200221 = (ee.Image('projects/sixth-firmament-425715-j4/assets/LeeMine_20200221')
    .clip(geometry_LeeMine).set('system:time_start',LeeMine_20200221_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
LeeMine_20200307_2 = (ee.Image('projects/sixth-firmament-425715-j4/assets/LeeMine_20200307_2')
    .clip(geometry_LeeMine).set('system:time_start',LeeMine_20200307_2_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
LeeMine_20210304 = (ee.Image('projects/sixth-firmament-425715-j4/assets/LeeMine_20210304')
    .clip(geometry_LeeMine).set('system:time_start',LeeMine_20210304_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
McConnel1_20190316 = (ee.Image('projects/sixth-firmament-425715-j4/assets/McConnel_20190316')
    .clip(geometry_McConnel1).set('system:time_start',McConnel1_20190316_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
McConnel2_20200307 = (ee.Image('projects/sixth-firmament-425715-j4/assets/McConnel_20200307')
    .clip(geometry_McConnel2).set('system:time_start',McConnel2_20200307_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
McConnel2_20210303 = (ee.Image('projects/sixth-firmament-425715-j4/assets/McConnel_20210303')
    .clip(geometry_McConnel2).set('system:time_start',McConnel2_20210303_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
OzarkHill_20170217_2 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Ozark_20170217_2')
    .clip(geometry_OzarkHill).set('system:time_start',OzarkHill_20170217_2_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
OzarkHill_20170407 = (ee.Image('projects/sixth-firmament-425715-j4/assets/OzarkHilll_20170407')
    .clip(geometry_OzarkHill).set('system:time_start',OzarkHill_20170407_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
OzarkHill_20180227 = (ee.Image('projects/sixth-firmament-425715-j4/assets/OzarkHilll_20180227')
    .clip(geometry_OzarkHill).set('system:time_start',OzarkHill_20180227_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
PennantBar1_20170205 = (ee.Image('projects/sixth-firmament-425715-j4/assets/PennantBar1_20170205')
    .clip(geometry_PennantBar1).set('system:time_start',PennantBar1_20170208_2_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
PennantBar1_20170216 = (ee.Image('projects/sixth-firmament-425715-j4/assets/PennantBar_20170216')
    .clip(geometry_PennantBar1).set('system:time_start',PennantBar1_20170216_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
PennantBar1_20180208_2 = (ee.Image('projects/sixth-firmament-425715-j4/assets/PennantBar1_20180208_2')
    .clip(geometry_PennantBar1).set('system:time_start',PennantBar1_20180212_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
SimpsonTownshipBarrens_PVT_20200229 = (ee.Image('projects/sixth-firmament-425715-j4/assets/SimpsonTownshipBarrens_PVT_20200229')
    .clip(geometry_SimpsonTownshipBarrens_PVT).set('system:time_start',SimpsonTownshipBarrens_PVT_20200229_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
TPL_20200119 = (ee.Image('projects/sixth-firmament-425715-j4/assets/TPL_20200119')
    .clip(geometry_TPL).set('system:time_start',TPL_20200119_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
TPL_20200227 = (ee.Image('projects/sixth-firmament-425715-j4/assets/TPL_20200227')
    .clip(geometry_TPL).set('system:time_start',TPL_20200227_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
TPL_20210205 = (ee.Image('projects/sixth-firmament-425715-j4/assets/TPL_20210205')
    .clip(geometry_TPL).set('system:time_start',TPL_20210205_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
White_20180227 = (ee.Image('projects/sixth-firmament-425715-j4/assets/White_20180227_2')
    .clip(geometry_White).set('system:time_start',White_20180227_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
White_20180306 = (ee.Image('projects/sixth-firmament-425715-j4/assets/White_20180306')
    .clip(geometry_White).set('system:time_start',White_20180306_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
White_20190322_2 = (ee.Image('projects/sixth-firmament-425715-j4/assets/White_20190322_2')
    .clip(geometry_White).set('system:time_start',White_20190322_2_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
WhoopieCat_20200307_2 = (ee.Image('projects/sixth-firmament-425715-j4/assets/WhoopieCat_20200307_2')
    .clip(geometry_WhoopieCat).set('system:time_start',WhoopieCat_20200307_2_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))

In [ ]:
def prep_HLS(image):
    """Scale, apply cloud/shadow mask, and select/rename Landsat 8 bands."""
    qa_mask = image.select('Fmask').bitwiseAnd(int('11111', 2)).eq(0)

    def get_factor_img(factor_names):
        factor_list = image.toDictionary().select(factor_names).values()
        return ee.Image.constant(factor_list)

    scaled = image.multiply(1)

    return image.addBands(scaled, None, True).select(
        ['B4', 'B3', 'B2', 'B5'], ['R', 'G', 'B', 'nir']).updateMask(qa_mask)

# Get the HLS collection, cloud masked and scaled to surface reflectance.
HLS_col_Ashby1 = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_Ashby1).map(prep_HLS)
HLS_col_Blowdown = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_Blowdown).map(prep_HLS)
HLS_col_JohnsonCo = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_JohnsonCo).map(prep_HLS)
HLS_col_LaRuePineHills = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_LaRuePineHills).map(prep_HLS)
HLS_col_LeeMine = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_LeeMine).map(prep_HLS)
HLS_col_McConnel1 = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_McConnel1).map(prep_HLS)
HLS_col_McConnel2 = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_McConnel2).map(prep_HLS)
HLS_col_OzarkHill = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_OzarkHill).map(prep_HLS)
HLS_col_PennantBar1 = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_PennantBar1).map(prep_HLS)
HLS_col_SimpsonTownshipBarrens_PVT = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_SimpsonTownshipBarrens_PVT).map(prep_HLS)
HLS_col_TPL = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_TPL).map(prep_HLS)
HLS_col_White = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_White).map(prep_HLS)
HLS_col_WhoopieCat = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_WhoopieCat).map(prep_HLS)

In [ ]:
#reference_Ashby1_20190209 = find_closest(Ashby1_20190209, HLS_col_Ashby1, 64).sort('CLOUD_COVERAGE').mosaic()
reference_Blowdown_20210122_2 = find_closest(Blowdown_20210122_2, HLS_col_Blowdown, 50).sort('CLOUD_COVERAGE').mosaic()
#reference_Blowdown_20210205_2 = find_closest(Blowdown_20210205_2, HLS_col_Blowdown, 32).sort('CLOUD_COVERAGE').mosaic()
reference_Blowdown_20220219_2 = find_closest(Blowdown_20220219_2, HLS_col_Blowdown, 50).sort('CLOUD_COVERAGE').mosaic()
#reference_JohnsonCo_20170216_2 = find_closest(JohnsonCo_20170216_2, HLS_col_JohnsonCo, 32).sort('CLOUD_COVERAGE').mosaic()
#reference_JohnsonCo_20170216_3 = find_closest(JohnsonCo_20170216_3, HLS_col_JohnsonCo, 32).sort('CLOUD_COVERAGE').mosaic()
#reference_JohnsonCo_20170308 = find_closest(JohnsonCo_20170308, HLS_col_JohnsonCo, 32).sort('CLOUD_COVERAGE').mosaic()
#reference_LaRuePineHills_20170217 = find_closest(LaRuePineHills_20170217, HLS_col_LaRuePineHills, 32).sort('CLOUD_COVERAGE').mosaic()
reference_LaRue_20170315_2 = find_closest(LaRue_20170315_2, HLS_col_LaRuePineHills, 32).sort('CLOUD_COVERAGE').mosaic()
reference_LaRue_20180306_2 = find_closest(LaRue_20180306_2, HLS_col_LaRuePineHills, 64).sort('CLOUD_COVERAGE').mosaic()
#reference_LeeMine_20200221 = find_closest(LeeMine_20200221, HLS_col_LeeMine, 32).sort('CLOUD_COVERAGE').mosaic()
reference_LeeMine_20200307_2 = find_closest(LeeMine_20200307_2, HLS_col_LeeMine, 32).sort('CLOUD_COVERAGE').mosaic()
#reference_LeeMine_20210304 = find_closest(LeeMine_20210304, HLS_col_LeeMine, 32).sort('CLOUD_COVERAGE').mosaic()
#reference_McConnel1_20190316 = find_closest(McConnel1_20190316, HLS_col_McConnel1, 32).sort('CLOUD_COVERAGE').mosaic()
#reference_McConnel2_20200307 = find_closest(McConnel2_20200307, HLS_col_McConnel1, 32).sort('CLOUD_COVERAGE').mosaic()
#reference_McConnel2_20210303 = find_closest(McConnel2_20210303, HLS_col_McConnel1, 32).sort('CLOUD_COVERAGE').mosaic()
reference_OzarkHilll_20170217_2 = find_closest(OzarkHill_20170217_2, HLS_col_OzarkHill, 32).sort('CLOUD_COVERAGE').mosaic()
#reference_OzarkHilll_20170407 = find_closest(OzarkHill_20170407, HLS_col_OzarkHill, 32).sort('CLOUD_COVERAGE').mosaic()
#reference_OzarkHilll_20180227 = find_closest(OzarkHill_20180227, HLS_col_OzarkHill, 32).sort('CLOUD_COVERAGE').mosaic()
#reference_PennantBar1_20170205 = find_closest(PennantBar1_20170205, HLS_col_PennantBar1, 32).sort('CLOUD_COVERAGE').mosaic()
#reference_PennantBar1_20170216 = find_closest(PennantBar1_20170216, HLS_col_PennantBar1, 32).sort('CLOUD_COVERAGE').mosaic()
reference_PennantBar1_20180208_2 = find_closest(PennantBar1_20180208_2, HLS_col_PennantBar1, 32).sort('CLOUD_COVERAGE').mosaic()
#reference_SimpsonTownshipBarrens_PVT_20200229 = find_closest(SimpsonTownshipBarrens_PVT_20200229, HLS_col_SimpsonTownshipBarrens_PVT, 32).sort('CLOUD_COVERAGE').mosaic()
#reference_TPL_20200119 = find_closest(TPL_20200119, HLS_col_TPL, 32).sort('CLOUD_COVERAGE').mosaic()
#reference_TPL_20200227 = find_closest(TPL_20200227, HLS_col_TPL, 32).sort('CLOUD_COVERAGE').mosaic()
#reference_TPL_20210205 = find_closest(TPL_20210205, HLS_col_TPL, 32).sort('CLOUD_COVERAGE').mosaic()
#reference_White_20180227 = find_closest(White_20180227, HLS_col_White, 32).sort('CLOUD_COVERAGE').mosaic()
#reference_White_20180306 = find_closest(White_20180306, HLS_col_White, 32).sort('CLOUD_COVERAGE').mosaic()
reference_White_20190322_2 = find_closest(White_20190322_2, HLS_col_White, 50).sort('CLOUD_COVERAGE').mosaic()
reference_WhoopieCat_20200307_2 = find_closest(WhoopieCat_20200307_2, HLS_col_WhoopieCat, 32).sort('CLOUD_COVERAGE').mosaic()

#result_Ashby1_20190209 = histogram_match(Ashby1_20190209, reference_Ashby1_20190209, geometry_Ashby1)
result_Blowdown_20210122_2 = histogram_match(Blowdown_20210122_2, reference_Blowdown_20210122_2, geometry_Blowdown)
result_Blowdown_20210205_2 = histogram_match(Blowdown_20210205_2, reference_Blowdown_20210205_2, geometry_Blowdown)
result_Blowdown_20220219_2 = histogram_match(Blowdown_20220219_2, reference_Blowdown_20220219_2, geometry_Blowdown)
result_JohnsonCo_20170216_2 = histogram_match(JohnsonCo_20170216_2, reference_JohnsonCo_20170216_2, geometry_JohnsonCo)
result_JohnsonCo_20170216_3 = histogram_match(JohnsonCo_20170216_3, reference_JohnsonCo_20170216_3, geometry_JohnsonCo)
result_JohnsonCo_20170308 = histogram_match(JohnsonCo_20170308, reference_JohnsonCo_20170308, geometry_JohnsonCo)
result_LaRuePineHills_20170217 = histogram_match(LaRuePineHills_20170217, reference_LaRuePineHills_20170217, geometry_LaRuePineHills)
result_LaRue_20170315_2 = histogram_match(LaRue_20170315_2, reference_LaRue_20170315_2, geometry_LaRuePineHills)
result_LaRue_20180306_2 = histogram_match(LaRue_20180306_2, reference_LaRue_20180306_2, geometry_LaRuePineHills)
result_LeeMine_20200221 = histogram_match(LeeMine_20200221, reference_LeeMine_20200221, geometry_LeeMine)
result_LeeMine_20200307_2 = histogram_match(LeeMine_20200307_2, reference_LeeMine_20200307_2, geometry_LeeMine)
result_LeeMine_20210304 = histogram_match(LeeMine_20210304, reference_LeeMine_20210304, geometry_LeeMine)
result_McConnel1_20190316 = histogram_match(McConnel1_20190316, reference_McConnel1_20190316, geometry_McConnel1)
result_McConnel2_20200307 = histogram_match(McConnel2_20200307, reference_McConnel2_20200307, geometry_McConnel1)
result_McConnel2_20210303 = histogram_match(McConnel2_20210303, reference_McConnel2_20210303, geometry_McConnel1)
result_OzarkHilll_20170217_2 = histogram_match(OzarkHill_20170217_2, reference_OzarkHilll_20170217_2, geometry_OzarkHill)
result_OzarkHilll_20170407 = histogram_match(OzarkHill_20170407, reference_OzarkHilll_20170407, geometry_OzarkHill)
result_OzarkHilll_20180227 = histogram_match(OzarkHill_20180227, reference_OzarkHilll_20180227, geometry_OzarkHill)
#result_PennantBar1_20170205 = histogram_match(PennantBar1_20170205, reference_PennantBar1_20170205, geometry_PennantBar1)
result_PennantBar1_20170216 = histogram_match(PennantBar1_20170216, reference_PennantBar1_20170216, geometry_PennantBar1)
result_PennantBar1_20180208_2 = histogram_match(PennantBar1_20180208_2, reference_PennantBar1_20180208_2, geometry_PennantBar1)
result_SimpsonTownshipBarrens_PVT_20200229 = histogram_match(SimpsonTownshipBarrens_PVT_20200229, reference_SimpsonTownshipBarrens_PVT_20200229, geometry_SimpsonTownshipBarrens_PVT)
result_TPL_20200119 = histogram_match(TPL_20200119, reference_TPL_20200119, geometry_TPL)
result_TPL_20200227 = histogram_match(TPL_20200227, reference_TPL_20200227, geometry_TPL)
result_TPL_20210205 = histogram_match(TPL_20210205, reference_TPL_20210205, geometry_TPL)
result_White_20180227 = histogram_match(White_20180227, reference_White_20180227, geometry_White)
result_White_20180306 = histogram_match(White_20180306, reference_White_20180306, geometry_White)
result_White_20190322_2 = histogram_match(White_20190322_2, reference_White_20190322_2, geometry_White)
result_WhoopieCat_20200307_2 = histogram_match(WhoopieCat_20200307_2, reference_WhoopieCat_20200307_2, geometry_WhoopieCat)

In [ ]:
# Define the region (Burna dn Bf area)
AOI = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/WhoopieCat_20200308_Poly')
AOI = AOI.geometry()

# Reduce the region. The region parameter is the Feature geometry.
image = result_WhoopieCat_20200307_2
region = AOI
# Reduce the region. The region parameter is the Feature geometry.
meanDictionary = image.reduceRegion(**{
  'reducer': ee.Reducer.min(),
  'geometry': region,
  'scale': 3,
  'maxPixels': 1e13
})
# The result is a Dictionary.  Print it.
print(meanDictionary.getInfo())

{'B': 0.0009765625, 'G': 0.01025390625, 'R': -0.026611328125, 'nir': -0.0074462890625}


In [ ]:
lon, lat, zoom = -89.46, 37.80 , 12
m = geemap.Map(center=[lat, lon], zoom=zoom)

vis_params_refl = {'min': 0, 'max': 0.25}
vis_params_dn = {'min': 0, 'max': 700}

m.add_layer(reference_WhoopieCat_20200307_2, vis_params_refl, 'Landsat-8 reference')
m.add_layer(WhoopieCat_20200307_2, vis_params_dn, 'Source')
m.add_layer(result_WhoopieCat_20200307_2, vis_params_refl, 'Matched')
m.add_layer(AOI, vis_params_refl, 'AOI')
m

Map(center=[37.8, -89.46], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [ ]:
task_config_Ashby1 = {'scale': 3,'region': geometry_Ashby1, 'maxPixels': 1e13}
task_config_Blowdown = {'scale': 3,'region': geometry_Blowdown, 'maxPixels': 1e13}
task_config_JohnsonCo = {'scale': 3,'region': geometry_JohnsonCo, 'maxPixels': 1e13}
task_config_LaRuePineHills = {'scale': 3,'region': geometry_LaRuePineHills, 'maxPixels': 1e13}
task_config_LeeMine = {'scale': 3,'region': geometry_LeeMine, 'maxPixels': 1e13}
task_config_McConnel1 = {'scale': 3,'region': geometry_McConnel1, 'maxPixels': 1e13}
task_config_McConnel2 = {'scale': 3,'region': geometry_McConnel2, 'maxPixels': 1e13}
task_config_OzarkHill = {'scale': 3,'region': geometry_OzarkHill, 'maxPixels': 1e13}
task_config_PennantBar1 = {'scale': 3,'region': geometry_PennantBar1, 'maxPixels': 1e13}
task_config_SimpsonTownshipBarrens_PVT = {'scale': 3,'region': geometry_SimpsonTownshipBarrens_PVT, 'maxPixels': 1e13}
task_config_TPL = {'scale': 3,'region': geometry_TPL, 'maxPixels': 1e13}
task_config_White = {'scale': 3,'region': geometry_White, 'maxPixels': 1e13}
task_config_WhoopieCat = {'scale': 3,'region': geometry_WhoopieCat, 'maxPixels': 1e13}

#task1 = ee.batch.Export.image(result_Ashby1_20190209, 'Ashby1_20190209', task_config_Ashby1)
task2 = ee.batch.Export.image(result_Blowdown_20210122_2, 'Blowdown_20210122_2', task_config_Blowdown)
#task3 = ee.batch.Export.image(result_Blowdown_20210205_2, 'Blowdown_20210205_2', task_config_Blowdown)
task4 = ee.batch.Export.image(result_Blowdown_20220219_2, 'Blowdown_20220219_2', task_config_Blowdown)
#task5 = ee.batch.Export.image(result_JohnsonCo_20170216_2, 'JohnsonCo_20170216_2', task_config_JohnsonCo)
#task6 = ee.batch.Export.image(result_JohnsonCo_20170216_3, 'JohnsonCo_20170216_3', task_config_JohnsonCo)
#task7 = ee.batch.Export.image(result_JohnsonCo_20170308, 'JohnsonCo_20170308', task_config_JohnsonCo)
#task8 = ee.batch.Export.image(result_LaRuePineHills_20170217, 'LaRuePineHills_20170217', task_config_LaRuePineHills)
task9 = ee.batch.Export.image(result_LaRue_20170315_2, 'LaRue_20170315_2', task_config_LaRuePineHills)
task10 = ee.batch.Export.image(result_LaRue_20180306_2, 'LaRue_20180306_2', task_config_LaRuePineHills)
#task11 = ee.batch.Export.image(result_LeeMine_20200221, 'LeeMine_20200221', task_config_LeeMine)
task12 = ee.batch.Export.image(result_LeeMine_20200307_2, 'LeeMine_20200307_2', task_config_LeeMine)
#task13 = ee.batch.Export.image(result_LeeMine_20210304, 'LeeMine_20210304', task_config_LeeMine)
#task14 = ee.batch.Export.image(result_McConnel1_20190316, 'McConnel1_20190316', task_config_McConnel)
#task15 = ee.batch.Export.image(result_McConnel2_20200307, 'McConnel2_20200307', task_config_McConnel)
#task16 = ee.batch.Export.image(result_McConnel2_20210303, 'McConnel2_20210303', task_config_McConnel)
task17 = ee.batch.Export.image(result_OzarkHilll_20170217_2, 'OzarkHilll_20170217_2', task_config_OzarkHill)
#task18 = ee.batch.Export.image(result_OzarkHilll_20170407, 'OzarkHilll_20170407', task_config_OzarkHilll)
#task19 = ee.batch.Export.image(result_OzarkHilll_20180227, 'OzarkHilll_20180227', task_config_OzarkHilll)
#task20 = ee.batch.Export.image(result_PennantBar1_20170205, 'PennantBar1_20170205', task_config_PennantBar1)
#task21 = ee.batch.Export.image(result_PennantBar1_20170216, 'PennantBar1_20170216', task_config_PennantBar1)
task22 = ee.batch.Export.image(result_PennantBar1_20180208_2, 'PennantBar1_20180208_2', task_config_PennantBar1)
#task23 = ee.batch.Export.image(result_SimpsonTownshipBarrens_PVT_20200229, 'SimpsonTownshipBarrens_PVT_20200229', task_config_SimpsonTownshipBarrens)
#task24 = ee.batch.Export.image(result_TPL_20200119, 'TPL_20200119', task_config_TPL)
#task25 = ee.batch.Export.image(result_TPL_20200227, 'TPL_20200227', task_config_TPL)
#task26 = ee.batch.Export.image(result_TPL_20210205, 'TPL_20210205', task_config_TPL)
#task27 = ee.batch.Export.image(result_White_20180227, 'White_20180227', task_config_White)
#task28 = ee.batch.Export.image(result_White_20180306, 'White_20180306', task_config_White)
task29 = ee.batch.Export.image(result_White_20190322_2, 'White_20190322_2', task_config_White)
task30 = ee.batch.Export.image(result_WhoopieCat_20200307_2, 'WhoopieCat_20200307_2', task_config_WhoopieCat)

#task1.start()
task2.start()
#task3.start()
task4.start()
#task5.start()
#task6.start()
#task7.start()
#task8.start()
task9.start()
task10.start()
#task11.start()
task12.start()
#task13.start()
#task14.start()
#task15.start()
#task16.start()
task17.start()
#task18.start()
#task19.start()
#task20.start()
#task21.start()
task22.start()
#task23.start()
#task24.start()
#task25.start()
#task26.start()
#task27.start()
#task28.start()
task29.start()
task30.start()

In [ ]:
def find_closest_clouds(target_image, image_col, days):
    range = ee.Number(days).multiply(1000 * 60 * 60 * 24)

    # Filter by intersection of geometries
    filter = ee.Filter.intersects('.geo', None, '.geo')

    # Join based on geometry intersection
    join = ee.Join.saveAll('matches').apply(ee.ImageCollection([target_image]), image_col, filter)

    # Convert the result into an ImageCollection
    matches = ee.ImageCollection(ee.List(join.first().get('matches')))

    # Sort by cloud cover
    sorted_by_cloud = matches.sort('CLOUD_COVER')

    # Filter the sorted collection by nearest date
    filter_date = ee.Filter.maxDifference(range, 'system:time_start', None, 'system:time_start')
    closest = ee.ImageCollection(sorted_by_cloud.filter(filter_date))

    return closest
